# Delsys FileReaderAPI - Python Overview
This document outlines how to directly read SHPF files within a python environment using Delsys' FileReaderAPI.
You can run this jupyter notebook example or export to a python script if desired.
    
## Python Packages (see requirements.txt)
1) ipykernel
2) pythonnet
3) numpy
4) plotly
5) pandas
6) nbformat

## How to export this notebook to a Python Script (if desired)

1) Open a terminal and run:
    ipython nbconvert --to python *.ipynb

2) delete:
    get_ipython().system('{sys.executable} -m pip install -r requirements.txt')

3) wrap the example in a main() function.

    def main():
        # all your code here

4) Add a __name__ function
    if __name__ == "__main__":
        # execute only if run as a script
        main()

# Python Example
## Install requirements that are noted in the requirement.txt file

In [ ]:
pip install -r requirements.txt

## Setup the File Reader

First import all dependencies and local python files. Then create the File Reader object

In [ ]:
import os
from Setup_Configuration import setup
setup(os.path.join(os.getcwd(), "dependencies"))
import glob
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from File_Reader import File_Reader
from DelsysFile import DelsysFile
from Component import Component
from Channel import Channel

#Read File
fileReader = File_Reader()

## Choosing a file to read
You can read a file by calling the .Read() method and passing the full path to the shpf file.
Calling the method ParsedFile() will return a file object that contains all the data from that file.

In [ ]:
filePath = os.path.join(os.getcwd(), "Forearm_Pronation_Supination_1.delsys")

fileReader.readFile(filePath)

fileReader.ParsedFile()


fileType = fileReader.FileType()

print("Loaded File: " + filePath)


## Navigating The File

You can loop through trials, the sensors within the trial, and the channels within the sensor, by using indexing

In [ ]:
gap = "—— "
for i in range(fileReader.TrialCount()):
    trial = fileReader.Trial(i)
    print("Trial: "+ trial.Name())
    
    for j in range(trial.ComponentCount()):
        component = trial.Component(j)
        print(gap + "Component: " + component.Name())
        
        for k in range(component.ChannelCount()):
            channel = component.Channel(k)
            print(gap + gap + "Channel: " + channel.Name())

## Retrieving Data
After the file is parsed all of the individual sensor component objects are stored in a list. The file provided with this example only has one sensor's data, therefore we will reference index 0 to retrieve the first component in that list. 

Get channel XY-data by calling GetChannelXyData(channel.Guid()) on the trial object

Get channel data by calling .yData on the xyData object.

Get channel time series data by calling .xData on xyData object.

In [ ]:
file = fileReader.Trial(0)
component1 = file.Component(0)
channel1 = component1.Channel(0)

xyData = file.GetChannelXyData(channel1.Guid())
channelydata = list(xyData.yData)
channelxdata = list(xyData.xData)

print("Trial Name: " + file.Name())
print("Component Name: " + component1.Name())

## Get all data from a component
You can get all channel data (for a single sensor component) and other metadata with these method calls.

In [ ]:
data = component1.GetAllData()
names = component1.GetAllChannelNames()
sampleRates = component1.GetAllSampleRates()
units = component1.GetAllUnits()

print("Component Channel Data:")
for i in range(len(data)):
    print("-- Channel: " + names[i] +
          "  Sample Rate: " + str(round(sampleRates[i], 2)) +
          "  Unit: " + units[i] +
          "  Data Length: " + str(len(data[i])))

## Get data from a single channel 
Each component object contains a list of channel objects. This example will pull the first channel from component1 object. Each channel object has a variety of metadata along with all of the channel data. To see all of the metadata and methods associated with each channel object see Channel.py 

In [ ]:
channel1 = component1.Channel(0)
print("Channel Name: " + channel1.Name())

## Plotting single channel of data
The single channel of data plotted on its own figure

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.update_layout(
title_text=channel1.Name()
)

#Single Channel PLot
fig.add_trace(go.Scattergl(
    x=channelxdata, 
    y=channelydata,
    mode="lines",
    name=channel1.Name()
))

# Set x-axis title
fig.update_xaxes(title_text="Time (s)"
)
# Set y-axes titles
fig.update_yaxes(title_text=channel1.Units()
)

## Plot all data from sensor component
Here all of the data is plotted based on the channel type (EMG, ACC, GYRO)

In [ ]:
#Loop through all channels and store each unique type in a list (EMG, ACC, GYRO)
channel_types = []
for i in range(component1.ChannelCount()):
    if component1.Channel(i).ChannelType() not in channel_types:
        channel_types.append(component1.Channel(i).ChannelType())

#Create a new plot for each channel type
fig = make_subplots(rows=len(channel_types), cols=1,
                    shared_xaxes=True,
)
fig.update_layout(
title_text="All Data"
)

#Loop all unique channel types
for i in range(len(channel_types)):

    #Loop all channel data - if channel type matches, add the channel data to that plot
    for k in range(component1.ChannelCount()):
        if component1.Channel(k).ChannelType() == channel_types[i]:
            #x-axes time values based on sampling rates
            xyData = file.GetChannelXyData(component1.Channel(k).Guid())
            data = list(xyData.yData)
            time = list(xyData.xData)
            #Add data
            fig.add_trace(go.Scatter(x=time, y=data,
                                mode="lines",
                                name=component1.Channel(k).Name()),
                                row=i+1,
                                col=1
            )
            # Set y-axes titles
            fig.update_yaxes(title_text=units[k],
                row=i+1,
                col=1
            )

fig.update_xaxes(title_text="Time (s)", row=len(channel_types), col=1)
fig.show()

## Close Files
Data from the files is dynamically loaded, meaning the files are left open until explicitly closed

In [ ]:
fileReader.Close()
print ("Closed")